In [1]:
import pandas as pd
import diversedata as dd

In [2]:
# List available datasets
dd.list_available_datasets()

Available datasets:

• bcindigenousbiz
  Load with:                load_data('bcindigenousbiz')
  View documentation with:  print_data_description('bcindigenousbiz')

• genderassessment
  Load with:                load_data('genderassessment')
  View documentation with:  print_data_description('genderassessment')

• globalrights
  Load with:                load_data('globalrights')
  View documentation with:  print_data_description('globalrights')

• hcmst
  Load with:                load_data('hcmst')
  View documentation with:  print_data_description('hcmst')

• wildfire
  Load with:                load_data('wildfire')
  View documentation with:  print_data_description('wildfire')

• womensmarchmadness
  Load with:                load_data('womensmarchmadness')
  View documentation with:  print_data_description('womensmarchmadness')



In [3]:
dd.load_data('hcmst')

,subject_age,subject_education,subject_sex,subject_ethnicity,subject_income_category,subject_employment_status,same_sex_couple,married,sex_frequency,flirts_with_partner,...,relationship_duration,children,rel_change_during_pandemic,inc_change_during_pandemic,subject_had_covid,partner_had_covid,subject_vaccinated,partner_vaccinated,agree_covid_approach,relationship_quality
0,53.0,high_school_grad,female,white,35k_40k,working_paid_employee,no,not_married,once_or_twice_a_week,a_few_times_a_week,...,1.500000,2.0,better_than_before,no_change,no,yes,not_vaccinated,not_vaccinated,completely_agree,excellent
1,72.0,some_college,female,white,75k_85k,working_paid_employee,no,married,once_a_month_or_less,never,...,57.416668,1.0,no_change,worse,no,no,fully_vaccinated_and_booster,fully_vaccinated_and_booster,mostly_agree,good
2,43.0,associate_degree,male,white,75k_85k,working_paid_employee,no,married,once_or_twice_a_week,a_few_times_a_week,...,22.333334,5.0,no_change,worse,no,no,fully_vaccinated_and_booster,fully_vaccinated_and_booster,completely_agree,excellent
3,64.0,some_college,male,white,75k_85k,working_paid_employee,no,married,once_or_twice_a_week,1_to_3_times_a_month,...,28.250000,2.0,no_change,no_change,no,no,fully_vaccinated_and_booster,fully_vaccinated_and_booster,completely_agree,good
4,60.0,high_school_grad,female,black,75k_85k,working_paid_employee,no,married,once_or_twice_a_week,a_few_times_a_week,...,38.916668,3.0,better_than_before,no_change,no,no,not_vaccinated,partially_vaccinated,completely_agree,excellent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,49.0,masters_degree,male,white,200k_250k,working_paid_employee,no,married,once_or_twice_a_week,1_to_3_times_a_month,...,25.583334,3.0,no_change,better,yes,no,fully_vaccinated_no_booster,fully_vaccinated_no_booster,completely_agree,good
1324,54.0,masters_degree,female,white,100k_125k,working_paid_employee,no,married,once_a_month_or_less,never,...,33.583332,2.0,no_change,worse,no,yes,fully_vaccinated_and_booster,fully_vaccinated_and_booster,completely_agree,good
1325,72.0,bach_degree,male,white,100k_125k,not_working_retired,no,married,once_a_month_or_less,a_few_times_a_week,...,41.083332,2.0,better_than_before,no_change,no,no,fully_vaccinated_and_booster,fully_vaccinated_no_booster,mostly_agree,excellent
1326,30.0,bach_degree,female,white,100k_125k,working_paid_employee,yes,not_married,once_or_twice_a_week,a_few_times_a_week,...,1.916667,2.0,no_change,much_worse,no,yes,fully_vaccinated_and_booster,fully_vaccinated_and_booster,completely_agree,good
